# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Мы обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.71. 

## Подготовка

### Импорт библиотек

In [1]:
!pip install transformers
!pip install catboost
!pip install optuna

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 636 kB 46.8 MB/s 
     |████████████████████████████████| 56 kB 5.6 MB/s 
     |████████████████████████████████| 895 kB 52.3 MB/s 
     |████████████████████████████████| 3.3 MB 46.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 76.4 MB 22 kB/s 
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 209 kB 52.1 MB/s 
     |████████████████████████████████| 80 kB 9.8 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 111 kB 58.6 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 144 kB 49.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=6f4a6bfd83eb5cd13

In [2]:
import numpy as np
import pandas as pd
import torch
import transformers
from transformers import DistilBertTokenizer, DistilBertModel
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import sklearn.svm

import optuna

from google.colab import files


In [3]:
files.upload()

Saving toxic_comments.csv to toxic_comments.csv


### Создадим токенайзер и модель

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

### Прочитаем таблицу

In [ ]:
df_tweets = pd.read_csv('toxic_comments.csv')
df_tweets.info()
df_tweets.head()

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

В таблице 2 столбца. Один столбец *text* с текстами комментаторов, который нам предстоит перевести в признаки для обучения и предсказания нашей модели, а второй столбец *toxic* содержит целевой признак. Что бы хотя бы немного ускорить работу программы, переведём данные в столбце *toxic* из int64 в uint8 

In [7]:
df_tweets['toxic'] = df_tweets['toxic'].astype('uint8')
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  uint8 
dtypes: object(1), uint8(1)
memory usage: 1.4+ MB


Выборку уменьшим, что бы не ждать бескнечно долго рассчётов дальнейшего эмбеддинга.

In [8]:
df_tweets = df_tweets.sample(2000, replace=True).reset_index(drop=True)

In [9]:
tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, add_special_tokens=True))

In [10]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

### Произведём эмбеддинг

In [11]:

batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/20 [00:00<?, ?it/s]

### Разделим таблицу на выборки

Объединим эмбеддинги, чтобы получить признаки для дальнейшей разбивки на тренировочную и тестовую выборки, а также получим целевой признак из столбца `df_tweets['toxic']`.

In [12]:
features = np.concatenate(embeddings)

target = df_tweets['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=12345)

## Обучим модели

### Модель логистической регрессии

In [13]:
model_log = LogisticRegression()
model_log.fit(features_train, target_train)
predictions = model_log.predict(features_test)
f1_log = f1_score(target_test, predictions)
print('F1 модели логистической регрессии:', '{:.2f}'.format(f1_log))

F1 модели логистической регрессии: 0.71


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



### Модель случайного леса

Для подбора гиперпараметров используем библиотеку *optuna*.

In [14]:
from sklearn.metrics import make_scorer

def objective(trial):
    rf_max_depth = int(trial.suggest_loguniform('rf_max_depth', 2, 20))
    rf_n_estimators = int(trial.suggest_loguniform('rf_n_estimators', 2, 80))
    classifier_obj = RandomForestClassifier(max_depth=rf_max_depth, n_estimators=rf_n_estimators, random_state=12345)
        
    score = cross_val_score(classifier_obj, features, target, scoring='f1', n_jobs=-1, cv=5)
    accuracy = score.mean()
    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    print(study.best_trial)

[I 2021-10-12 12:08:24,178] A new study created in memory with name: no-name-f018bbd9-2e66-4847-8604-b1faa4676a22
[I 2021-10-12 12:08:27,720] Trial 0 finished with value: 0.06076923076923078 and parameters: {'rf_max_depth': 3.8529589506472277, 'rf_n_estimators': 14.39250942986849}. Best is trial 0 with value: 0.06076923076923078.
[I 2021-10-12 12:08:30,897] Trial 1 finished with value: 0.2635113799038927 and parameters: {'rf_max_depth': 5.0148200918438155, 'rf_n_estimators': 44.160374655626434}. Best is trial 1 with value: 0.2635113799038927.
[I 2021-10-12 12:08:33,726] Trial 2 finished with value: 0.39308503988046184 and parameters: {'rf_max_depth': 12.361205666174508, 'rf_n_estimators': 22.9579286554159}. Best is trial 2 with value: 0.39308503988046184.
[I 2021-10-12 12:08:36,757] Trial 3 finished with value: 0.286172019195275 and parameters: {'rf_max_depth': 8.798526692404405, 'rf_n_estimators': 29.899599760554782}. Best is trial 2 with value: 0.39308503988046184.
[I 2021-10-12 12:0

FrozenTrial(number=2, values=[0.39308503988046184], datetime_start=datetime.datetime(2021, 10, 12, 12, 8, 30, 899602), datetime_complete=datetime.datetime(2021, 10, 12, 12, 8, 33, 725872), params={'rf_max_depth': 12.361205666174508, 'rf_n_estimators': 22.9579286554159}, distributions={'rf_max_depth': LogUniformDistribution(high=20.0, low=2.0), 'rf_n_estimators': LogUniformDistribution(high=80.0, low=2.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)


Лучший результат с гиперпараметрами max_depth=16, rf_n_estimators=46.

In [15]:
model_forest = RandomForestClassifier(max_depth=16, n_estimators=37, random_state=12345)
model_forest.fit(features_train, target_train)
predictions_forest = model_forest.predict(features_test)
f1_forest = f1_score(target_test, predictions_forest)
print('F1 модели случайного леса:', '{:.2f}'.format(f1_forest))

F1 модели случайного леса: 0.41


### Модель LightGBM

In [16]:
model_gbm = LGBMClassifier(random_state=12345, class_weight='balanced')
model_gbm.fit(features_train, target_train, verbose=10)

predictions_gbm = model_gbm.predict(features_test)
f1_gbm = f1_score(target_test, predictions_gbm)

print('F1 модели градиентного бустинга библиотеки lightGBM:', '{:.2f}'.format(f1_gbm))

F1 модели градиентного бустинга библиотеки lightGBM: 0.64


### Модель CatBoost

In [17]:
model_cat = CatBoostClassifier(iterations=100, random_seed=12345)
model_cat.fit(features_train, target_train, verbose=10)

predictions_cat = model_cat.predict(features_test)
f1_cat = f1_score(target_test, predictions_cat)

print('F1 модели градиентного бустинга библиотеки CatBoost:', '{:.2f}'.format(f1_cat))

Learning rate set to 0.09825
0:	learn: 0.5833254	total: 394ms	remaining: 39s
10:	learn: 0.1952058	total: 2.48s	remaining: 20.1s
20:	learn: 0.1171398	total: 4.58s	remaining: 17.3s
30:	learn: 0.0812995	total: 6.67s	remaining: 14.8s
40:	learn: 0.0629088	total: 8.75s	remaining: 12.6s
50:	learn: 0.0479983	total: 10.8s	remaining: 10.4s
60:	learn: 0.0401885	total: 12.9s	remaining: 8.24s
70:	learn: 0.0340603	total: 15s	remaining: 6.11s
80:	learn: 0.0274463	total: 17s	remaining: 4s
90:	learn: 0.0221400	total: 19.1s	remaining: 1.89s
99:	learn: 0.0186123	total: 21s	remaining: 0us
F1 модели градиентного бустинга библиотеки CatBoost: 0.60


## Вывод

Для того, чтобы определить с помощью моделей машинного обучения какие комментарии являются токсичными, мы создали токенайзер и использовали метод разбора текста из таблицы на эмбеддинги. После тренировки нескольких моделей, **лучшей из них оказалась модель логистической регрессии**, худшей модель случайного леса.